In [45]:
import pandas as pd


# Get Annotated files

In [46]:
!ls ../../data

Annotated_Anu_samples.csv
Annotated_Anu_samples.tsv
annotated_mel_samples.2023-04-30_23.17.25.824926.tsv
annotated_sarik_samples.2023-04-29_00.17.22.728471.tsv
anu_samples_20230428.204233.tsv
five_models_normalized_categorical_2023-04-17_23.29.43.888250.json
five_models_normalized_categorical_2023-04-17_23.29.43.888250.tsv
five_models_normalized_data_2023-04-17_22.57.12.712880.json
five_models_normalized_data_2023-04-17_22.57.12.712880.tsv
mel_samples_20230428.204233.csv
mel_samples_20230428.204233.tsv
mel_samples_20230428.204233.tsv.bak
mel_samples_20230428.204233_rescue.tsv
misc_intermediate
sarik_samples_20230428.204233.tsv


In [47]:
anu = '../../data/Annotated_Anu_samples.csv'
anu_df = pd.read_csv(anu)
anu_df.columns
anu_df.loc[anu_df['our_label'] == 'neu', 'our_label'] = 'neutral'
anu_df.loc[anu_df['our_label'] == 'neg', 'our_label'] = 'negative'
anu_df.loc[anu_df['our_label'] == 'pos', 'our_label'] = 'positive'
anu_df['annotator'] = 'anu'
anu_df[['our_label','annotator']].head()
anu_df['our_label'].value_counts()

our_label
neutral     162
negative     95
positive     43
Name: count, dtype: int64

In [48]:
sarik = '../../data/annotated_sarik_samples.2023-04-29_00.17.22.728471.tsv'
sarik_df = pd.read_csv(sarik, sep='\t')
sarik_df.loc[sarik_df['our_label'] == 'Neutral', 'our_label'] = 'neutral'
sarik_df.loc[sarik_df['our_label'] == 'Negative', 'our_label'] = 'negative'
sarik_df.loc[sarik_df['our_label'] == 'Positive', 'our_label'] = 'positive'
sarik_df['annotator'] = 'sarik'

sarik_df[['our_label','annotator']].head()
sarik_df['our_label'].value_counts()

our_label
negative    137
neutral     112
positive     51
Name: count, dtype: int64

In [50]:
#melissa = '../../data/mel_samples_20230428.204233.tsv.bak'
melissa = '../../data/annotated_mel_samples.2023-04-30_23.17.25.824926.tsv' # thank you Sarik!
mel_df = pd.read_csv(melissa, sep='\t')

print(mel_df.shape)
display(mel_df[['conversation_id','our_label','annotator']].head())
mel_df['our_label'].value_counts()

(300, 46)


,conversation_id,our_label,annotator
0,1633665401883332608,negative,melissa
1,1633965977665966080,neutral,melissa
2,1633219223626432512,neutral,melissa
3,1633874107552108544,neutral,melissa
4,1633227958260473856,neutral,melissa


our_label
neutral     129
negative    112
positive     59
Name: count, dtype: int64

# Combine our labels

In [51]:
df = pd.merge(mel_df, sarik_df[['our_label', 'conversation_id']], how='inner', on='conversation_id', suffixes=('_mel', '_sarik'))


In [53]:
assert df.shape[0] == 100, f'Wrong shape of merge: {df.shape}'
df[['conversation_id', 'our_label_mel', 'our_label_sarik']].head()

,conversation_id,our_label_mel,our_label_sarik
0,1633665401883332608,negative,negative
1,1633965977665966080,neutral,neutral
2,1633219223626432512,neutral,negative
3,1633874107552108544,neutral,neutral
4,1633227958260473856,neutral,negative


In [57]:
iaa_df = pd.merge(df, anu_df[['our_label', 'conversation_id']], how='inner', on='conversation_id')
assert iaa_df.shape[0] == 100, f'Wrong shape of iaa_df: {iaa_df.shape}'

# rename the final labels from Anu:
iaa_df['our_label_anu'] = iaa_df['our_label']

# reset the final "our_label" field to none, will be our final vote
iaa_df['our_label'] = ''
print(iaa_df.columns)

iaa_df[['conversation_id', 'our_label_mel', 'our_label_sarik', 'our_label_anu', 'our_label']].head()


Index(['Unnamed: 0', 'conversation_id', 'lang', 'reply_settings', 'created_at',
       'clean_text', 'text', 'author_id', 'referenced_tweets', 'id',
       'edit_history_tweet_ids', 'public_metrics.retweet_count',
       'public_metrics.reply_count', 'public_metrics.like_count',
       'public_metrics.impression_count', 'in_reply_to_user_id',
       'geo.place_id', 'withheld.copyright', 'withheld.country_codes',
       'geo.coordinates.type', 'geo.coordinates.coordinates',
       'textblob_Polarity', 'textblob_Subjectivity', 'vader_Polarity',
       'vader_Subjectivity', 'AFINN_scores', 'AFINN_sentiments',
       'SentiWordNet_scores', 'normalized_textblob_Polarity',
       'normalized_textblob_Subjectivity', 'normalized_vader_Polarity',
       'normalized_vader_Subjectivity', 'normalized_AFINN_scores',
       'normalized_SentiWordNet_scores', 'our_label_mel', 'annotator',
       'roberta_content', 'roberta_polarity', 'roberta_sentiment', 'keyword',
       'normalized_roberta_polarity'

,conversation_id,our_label_mel,our_label_sarik,our_label_anu,our_label
0,1633665401883332608,negative,negative,negative,
1,1633965977665966080,neutral,neutral,neutral,
2,1633219223626432512,neutral,negative,neutral,
3,1633874107552108544,neutral,neutral,neutral,
4,1633227958260473856,neutral,negative,neutral,


# Votes

In [67]:
annot_cols = ['our_label_mel', 'our_label_sarik', 'our_label_anu']
iaa_df['our_label'] = iaa_df[annot_cols].mode(axis=1)
display(iaa_df[['our_label_mel', 'our_label_sarik', 'our_label_anu', 'our_label']].iloc[:30])
display(iaa_df[['our_label_mel', 'our_label_sarik', 'our_label_anu', 'our_label']].iloc[30:60])
display(iaa_df[['our_label_mel', 'our_label_sarik', 'our_label_anu', 'our_label']].iloc[60:])

,our_label_mel,our_label_sarik,our_label_anu,our_label
0,negative,negative,negative,negative
1,neutral,neutral,neutral,neutral
2,neutral,negative,neutral,neutral
3,neutral,neutral,neutral,neutral
4,neutral,negative,neutral,neutral
5,neutral,neutral,neutral,neutral
6,neutral,neutral,neutral,neutral
7,neutral,neutral,neutral,neutral
8,neutral,negative,neutral,neutral
9,negative,negative,negative,negative


,our_label_mel,our_label_sarik,our_label_anu,our_label
30,neutral,negative,negative,negative
31,negative,negative,neutral,negative
32,neutral,negative,neutral,neutral
33,neutral,neutral,neutral,neutral
34,neutral,neutral,neutral,neutral
35,positive,positive,positive,positive
36,negative,neutral,neutral,neutral
37,positive,neutral,positive,positive
38,neutral,neutral,neutral,neutral
39,positive,positive,positive,positive


,our_label_mel,our_label_sarik,our_label_anu,our_label
60,positive,positive,positive,positive
61,positive,positive,positive,positive
62,positive,positive,neutral,positive
63,neutral,negative,neutral,neutral
64,neutral,neutral,neutral,neutral
65,positive,neutral,neutral,neutral
66,positive,positive,neutral,positive
67,negative,negative,negative,negative
68,negative,negative,neutral,negative
69,negative,negative,negative,negative


In [68]:
iaa_df['our_label'].value_counts()

our_label
neutral     47
negative    33
positive    20
Name: count, dtype: int64

# IAA

In [70]:
from sklearn.metrics import cohen_kappa_score as cohen
from itertools import combinations

In [82]:
annot_cols = ['our_label_mel', 'our_label_sarik', 'our_label_anu', 'our_label']
print('Our combinations (including the vote):\n', [x for x in combinations(annot_cols, 2)])
for pair in [x for x in combinations(annot_cols, 2)]:
    print(pair, cohen(iaa_df[pair[0]], iaa_df[pair[1]]))



Our combinations (including the vote):
 [('our_label_mel', 'our_label_sarik'), ('our_label_mel', 'our_label_anu'), ('our_label_mel', 'our_label'), ('our_label_sarik', 'our_label_anu'), ('our_label_sarik', 'our_label'), ('our_label_anu', 'our_label')]
('our_label_mel', 'our_label_sarik') 0.38214396045721355
('our_label_mel', 'our_label_anu') 0.48594377510040165
('our_label_mel', 'our_label') 0.7186181022354228
('our_label_sarik', 'our_label_anu') 0.4392523364485982
('our_label_sarik', 'our_label') 0.6578538102643858
('our_label_anu', 'our_label') 0.7687861271676301
